memory를 chain에 꽂는 방법과 두 종류의 chain을 사용해서 꽂는 방법을 배워보자.

첫 번째로 LLM Chain이라는 걸 사용해 본다.  

LLM chain 은 off-the-shelf chain 인데 (off-the-shelf: 바로 살 수 있는 기성품 느낌) 

off-the-shelf는 일반적인 목적을 가진 chain을 의미한다.  

만약 우리가 커스텀해서 만든 chain을 활용한다면 off-the-shlf chain 은 적합하지 않다.


In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
     llm=llm,
     max_token_limit=80
)

ConversationSummaryBufferMemory은 interaction(상호작용, 대화)들을 기본적으로 가지고 있는 conversation class 이다.  

그리고 interaction의 토큰 수가 max_token_limit(위 경우는 80개) 보다 많으면 가장 오래된 interaction을 요약해준다.  

즉, 최신 내용을 그대로 유지하고 대화 기록을 요약한다.

chain을 만들어 보자.

In [10]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
)

chain.predict(question="My name is Nico")

'Nice to meet you, Nico! How can I assist you today?'

In [5]:
chain.predict(question="I live in Seoul")

"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [6]:
chain.predict(question="what is my name?")

"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

보다시피 대화가 이어지도록 동작하지 않는다.  

위 chain을 디버깅 해 보자.  

langchain은 verbose 라는 아주 좋은 속성을 가지고 있다.  

이 값을 chain에 전달하면 chain을 실행했을 때 프롬프트 로그들을 확인할 수 있다.

In [11]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True
)

In [12]:
chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [13]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [14]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
what is my name?

> Finished chain.


"I'm sorry, I do not have access to that information."

위 내용을 살펴보면 대화의 내역이 프롬프트에 계속 추가되지 않는다.

우리가 원하는 어떤 방식으로 프롬프트에 대화 기록을 추가해 줘야 한다.

그런데 프롬프트와 별개로 memory 에는 계속 업데이트가 된다.

In [15]:
memory.load_memory_variables({})

{'history': 'System: Nico introduces themselves and the AI responds warmly. Nico mentions living in Seoul, and the AI expresses enthusiasm for the vibrant city, highlighting its rich history, culture, and various attractions. The AI asks Nico what they enjoy most about living in Seoul. The human asks what their name is, and the AI explains that it does not have access to personal information. The human asks if the AI can respond in Korean, to which the AI confirms and encourages them to ask any questions they may have in Korean. The human asks, "내 이름이 뭐야?" The AI apologizes and explains that it is an AI chatbot without a name, then asks for Nico\'s name. Nico responds, "My name is Nico." The AI warmly greets Nico and asks how it can assist them today. The AI expresses excitement about Seoul and all it has to offer, prompting Nico to share what they enjoy most about living in the vibrant city.\nHuman: what is my name?\nAI: I\'m sorry, I do not have access to that information.'}

위에 요약한 내용을 보다시피, 우리의 대화를 요약하는 건 잘 작동하고 있다.  

문제는 이것들은 프롬프트에 포함되지 않고 있다는 것이다.

일단 모든게 잘 동작하고 있긴 한 것 이지만...  메모리도 업데이트 되고 있고, 요약도 진행되고 있지만,  

문제는 프롬프트에 전달되어야 하는 대화 내용이 LLM에게 전달되지 않고  

지금 프롬프트는 우리에게 혹은 LLM에게 question만 주고 있다는 것이다.


이를 해결하기 위해서 template를 작성해 준다.  

template에서 chat_history를 가져와서 human에 대한 question을 작성하고 

you를 작성해서 AI가 우리의 대화 기록을 기억하여 여기에 question을 완성할 수 있기를 바래보자.

그리고 작성한 template를 LLMChain의 prompt에 넣어보자.

In [17]:
template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You:  
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

위에서 보다시피 history와 memory가 human과 LLM의 인풋 아웃풋을 기반으로 업데이트 되고 있다.  

이제 우리는 이 conversation memory class에게 템플릿 안에 콘텐츠를 넣으라고 얘기해 줘야 한다. 

이건 memory_key를 불러서 chat_history라고 말해주면 된다.

그리고 template 안에 memory가 history를 저장하도록 한 곳을 적어준다. {chat_histry}    

그럼 memory는 template안에 대화 기록을 저장하는 공간을 찾을 거고, 자동으로 그 메모리의 기록을 여기{chat_history}에 넣게 된다.

자 세팅됐으면 첫 번째 구문만 실행시켜보자.

In [19]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You:  
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is Nico
    You:  


> Finished chain.


'Hello Nico! How can I assist you today?'

In [20]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human: I live in Seoul
    You:  


> Finished chain.


"That's great to hear! How can I assist you with living in Seoul today?"

위 기록을 살펴보면 프롬프트에 기록이 남겨져 있는걸 확인할 수 있다.

마지막 구문을 실행시켜 보자.

In [21]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with living in Seoul today?
    Human: what is my name?
    You:  


> Finished chain.


'Your name is Nico.'

이제 우리는 memory를 우리의 아름다운 AI에 전달할 수 있게 되었다.


요약하자면, 우리는 그저 우리의 프롬프트 템플릿 안에 공간을 만들기만 하면 된다.  

우린 프롬프트 템플릿에 memory를 위한 공간을 만들고  

BufferMemory 등 우리가 쓰고 있는 memory class에게 history를 넣을 곳을 알려주기만 하면 된다.


요약도 잘 되는지 확인해보자.

In [22]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You:  
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is Nico
    You:  


> Finished chain.


'Hello Nico! How can I assist you today?'

In [23]:
chain.predict(question="I live in Seoul")
chain.predict(question="I live in Seoul")
chain.predict(question="I live in Seoul")
chain.predict(question="I live in Seoul")
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human: I live in Seoul
    You:  


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! Is there anything specific you would like help with regarding Seoul or anything else?
    Human: I live in Seoul
    You:  


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! Is there anything specific you would like help with regarding Seoul or anything else?
Human: I live in Seoul
AI: Yes, you mentioned that earlier. Is there

'It seems like you may be emphasizing the fact that you live in Seoul. Is there a particular aspect of living in Seoul that you would like to discuss or ask about? Feel free to share any specific questions or topics you have in mind!'

In [24]:
chain.predict(question="what is my name?")
chain.predict(question="what is my name?")
chain.predict(question="what is my name?")
chain.predict(question="what is my name?")
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: Nico introduces themselves and mentions they live in Seoul. The AI greets Nico and asks how it can assist, specifically in relation to Seoul. The AI notices Nico mentioning living in Seoul multiple times and asks if there is something specific they would like to discuss or ask about living in Seoul.
AI: It seems like you may be repeating yourself. Is there a particular reason why you keep mentioning that you live in Seoul? If there's something specific you'd like to discuss or ask about living in Seoul, feel free to let me know!
Human: I live in Seoul
AI: It seems like you may be emphasizing the fact that you live in Seoul. Is there a particular aspect of living in Seoul that you would like to discuss or ask about? Feel free to share any specific questions or topics you have in mind!
    Human: what is my name?
    You:  


> Finished chain.


> Entering new LLMCha

"I'm sorry, but I don't have access to personal information such as your name. If there's anything else you'd like to know or discuss, feel free to ask!"

위에서 보이는 거 같이 요약도 아주 잘 된다.  

프롬프트에도 아주 잘 표시되고.  

이게 우리가 memory에 추가하고 싶을 때  

문자열 형태로 된 프롬프트 템플릿에 대화 기록을 추가하고 싶을 때 하는 일이다.  

```python

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You:  
"""

```

이번에는 대화 기록을 추가하는 방식인데 메세지를 기반으로 한 사람과 AI의 대화 기록을 추가하는 방법을 알아보자.

대화 기반 메세지의 memory는 쉬운 편이다.

기억해야할 건 memory 클래스가 memory를 두 가지 방식으로 출력할 수 있다는 점이다.  
- 문자열 형태일 수도 있고, 
- message 형태일 수도 있다.

예를 들어 위에서 동작한 코드의 memory.load_memory_variables({})를 해보면 그냥 텍스트가 나오는걸 볼 수 있다.

In [25]:
memory.load_memory_variables({})

{'chat_history': "System: Nico introduces themselves and mentions they live in Seoul. The AI greets Nico and asks how it can assist, specifically in relation to Seoul. The AI notices Nico mentioning living in Seoul multiple times and asks if there is something specific they would like to discuss or ask about living in Seoul. The AI suggests that Nico may be repeating themselves and encourages them to share any specific questions or topics related to living in Seoul. The AI prompts Nico to share any particular aspects of living in Seoul they would like to discuss or ask about. The human asks the AI for their name, but the AI explains it doesn't have access to personal information like names and encourages the human to ask any other questions they may have. The human asks the AI for their name, but the AI reiterates that it doesn't have access to personal information and invites the human to ask any other questions they may have. The human asks the AI for their name, but the AI responds 

대화 기반의 채팅으로 사용하고 싶다면 메세지로 바꿔달라는 설정을 준다. (return_messages=True)  

이건 문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 걸 의미한다.  

[사람 메세지든, AI 메세지든, 시스템 메세지든] 실제 message 클래스로 바꿔달라고

In [26]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)


그런 다음 문자열 기반 템플릿 대신 우린 ChatPromptTemplate를 불러와 사용해보자.

In [27]:
from langchain.prompts import ChatPromptTemplate

# 문자열 기반 템플릿 지우고고
# template = """
#     You are a helpful AI talking to a human.
    
#     {chat_history}
#     Human: {question}
#     You:  
# """

# 대화 기반 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful AI talking to a human"),
    ("human", "{question}"),
])



그런데 이 question에 우리는 어떻게 대화 기록들을 넣어둘 수 있을까?  

대화 기록에는 엄청 많은 사람 메세지와 AI 메세지들이 있을 것이고 우리가 ConversationBufferMemory로부터 요약본을 받아올 때 시스템메세지도 추가될 예정이라서   

휴먼, AI, 휴먼, AI, 휴먼, AI, 휴먼, AI 메세지들이 있고 그 다음에 시스템 메세지가 있을거고 다시 휴먼, AI, 휴먼, AI 메세지 들이 있게 될 것이다.

이 많은 것들을 위한 공간을 우린 어떻게 만들 수 있을까???  

정답은 <b>MessagePlaceholder</b>을 활용하는 것이다!!

prompt에 MessagePlaceholder() 를 불러주고, 누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질 수 있다.

그럼 우리의 대화 요약 memory는 그 기록들에서 메세지들을 가져와서 모든 메세지 기록들로 MessagesPlaceholder를 채울 것이다.

In [28]:
from langchain.prompts import MessagesPlaceholder

# 대화 기반 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])



말했듯이 ConversationSummaryBufferMemory가 AI, Human, system 메세지를 줄거고,  

우리는 그게 얼마나 많은지 알 수 없기에 MessagesPlaceholder를 불렀고,  

얘네가 하는 역할은 우리는 메세지가 얼마나 많고 누구에게로부터 왔는지 모르지만 얘네가 이 memory class로 대체될 것이다.


In [29]:

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [30]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)


# 대화 기반 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [31]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?"

In [32]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

이번에는 어떻게 memory를 추가할 수 있을지 배워보자.  
(수동으로 만들어진 chain에 langchain expression 언어를 활용해서 추가하는 방법)  

우린 메모리를 문자열 기반 완성품들과 채팅 메세지 기반 완성품들에 넣는 방법을 배웠다.

다음은 커스텀 체인에 대한 설명이다.

LangChain Expression 언어를 이용하여 생성된 체인에 메모리를 추가하는 것은 어렵지 않고, 실제로 변경 작업을 할 때 권장되는 방법이다.

LLMChain 을 사용하는 대신 체인을 만들어 보자.

접근 방식 1



In [59]:
chain = prompt | llm 

chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question":"My name is Nico",
})

AIMessage(content='I apologize for the confusion earlier. It seems like you were confirming your name. Thank you for letting me know, Nico! How can I assist you further?')

이 접근방식의 문제는 우리가 체인을 호출할 때마다 chat_history도 추가해줘야 한다는 것.

---


접근 방식 2

RunnablePassthrough 를 사용한다.
```python
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
```
RunnablePassthrough.assign이 실행될 때 가장 먼저 load_memory 함수를 호출해서 리턴값을 prompt가 필요로 하는 chat_history 내부에 넣는다.

RunnablePassthrough가 하는 일?
prompt가 format되기 전에 함수를 실행시키는 걸 허락해준다.

RunnablePassthrough.assign 함수에서 우리가 원하는 어떤 값이든 변수에 할당할 수 있다.  
그리고 그 변수들은 prompt에 주어진다.


load_memory 함수는 argument 하나를 받아야 한다. 
왜냐하면 RunnablePassthrough.assign() 할 때 사용자의 input을 받기 때문.  

체인에 있는 모든 컴포넌트는 input을 받을거고, 또 output을 줄거다.

실행규칙) 모든 것은 input을 얻을거고, 그 후엔 output을 줄거다.
```python
chain.invoke(
    {
        "question": "My name is Nico",
    }
)
```
딕셔너리는 체인의 첫 번째 아이템의 input이 된다.

이제 체인에는 메모리가 있고, 우리가 여기서 하는건 load_memory 함수를 실행시키고,

load_memory 함수의 output은 chat_history의 키를 사용하는 프롬프트에 주어져야 한다고 말하는 것.

다시, load_memory 함수를 로드시키거나 실행시키고, 그 함수를 실행시키고 얻은 output은 

chat_history라는 속성으로 들어간다.

그리고 그게 사용자의 input과 결합돼서 프롬프트에 전달될것이다.

질문은 사용자에게서 왔고, chat_history는 RunnablePassthrough.assign() 코드를 실행한 결과에서 왔다.







In [47]:
from langchain.schema.runnable import RunnablePassthrough


def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

chain.invoke(
    {
        "question": "My name is Nico",
    }
)

AIMessage(content='Thank you, Nico! How can I assist you today?')

여기서 문제는 우리가 각 결과를 메모리에 저장해야 한다는 것이다.

왜냐하면 지금 메모리 관리를 수동적인 방법으로 하고 있기 때문에.

여기서 제일 좋은 방법은 체인을 호출하는 함수를 만드는 것.

In [48]:
def invoke_chain(question):
    result = chain.invoke(
        {"question": "My name is Nico"},
    )
    # input: 사용자의 질문, output: 그에 대한 결과
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

전체 코드

In [55]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

# 대화 기반 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


def invoke_chain(question):
    result = chain.invoke(
        {"question": question},
    )
    # input: 사용자의 질문, output: 그에 대한 결과
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

In [56]:
invoke_chain("My name is nico")


content='Nice to meet you, Nico! How can I assist you today?'


In [57]:
invoke_chain("I live in korea at incheon")

content="That's great to know, Nico! Is there anything specific you would like to know or talk about regarding Incheon or Korea?"


In [58]:
invoke_chain("just answer. What is my name?")

content='Your name is Nico.'
